# Respond in a format

The typical ReAct agent prompts the LLM to respond in 1 of two formats: a
function call (~ JSON) to use a tool, or conversational text to respond to the
user.

If your agent is connected to a structured (or even generative) UI, or if it is
communicating with another agent or software process, you may want it to resopnd
in a specific structured format.

In this example we will build a conversational ReAct agent that responds in a
specific format. We will do this by using
[tool calling](https://js.langchain.com/docs/modules/model_io/models/chat/function-calling/).
This is useful when you want to enforce that an agent's response is in a
specific format. In this example, we will ask it respond as if it were a
weatherman, returning the temperature and additional info in separate,
machine-readable fields.

## Setup

First we need to install the packages required

```bash
yarn add langchain @langchain/anthropic @langchain/langgraph
```


Next, we need to set API keys for Anthropic (the LLM we will use).


In [ ]:
// Deno.env.set("ANTHROPIC_API_KEY", "sk_...");


Optionally, we can set API key for
[LangSmith tracing](https://smith.langchain.com/), which will give us
best-in-class observability.

In [ ]:
// Deno.env.set("LANGCHAIN_API_KEY", "ls...");
Deno.env.set("LANGCHAIN_CALLBACKS_BACKGROUND", "true");
Deno.env.set("LANGCHAIN_TRACING_V2", "true");
Deno.env.set("LANGCHAIN_PROJECT", "Respond in Format: LangGraphJS");


## Set up the State


In [11]:
const graphState = {
  messages: {
    value: (x, y) => x.concat(y),
    default: () => [],
  },
};


## Set up the tools


In [1]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = new DynamicStructuredTool({
  name: "search",
  description: "Call to surf the web.",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
  func: async ({ query }: { query: string }) => {
    // This is a placeholder, but don't tell the LLM that...
    return "The answer to your question lies within.";
  },
});

const tools = [searchTool];


We can now wrap these tools in a simple
[ToolNode](https://langchain-ai.github.io/langgraphjs/reference/classes/prebuilt.ToolNode.html).

In [4]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const toolNode = new ToolNode(tools);


## Set up the model


In [5]:
import { ChatAnthropic } from "@langchain/anthropic";

const model = new ChatAnthropic({
  temperature: 0,
  model: "claude-3-haiku-20240307",
});


After we've done this, we should make sure the model knows that it has these
tools available to call. We can do this by converting the LangChain tools into
the format for function calling, and then bind them to the model class.

We also want to define a response schema for the language model and bind it to
the model as a function as well.

In [7]:
import { z } from "zod";

const Response = z.object({
  temperature: z.number().describe("the temperature"),
  other_notes: z.string().describe("any other notes about the weather"),
});

const boundModel = model.bindTools([
  ...tools,
  { name: "Response", schema: Response },
]);


## Define the nodes


In [9]:
import { BaseMessage } from "@langchain/core/messages";
import { AIMessage } from "@langchain/core/messages";

// Define the function that determines whether to continue or not
const route = (state: { messages: BaseMessage[] }) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1] as AIMessage;
  // If there is no function call, then we finish
  if (!lastMessage.tool_calls || lastMessage.tool_calls.length === 0) {
    return "end";
  }
  // Otherwise if there is, we need to check what type of function call it is
  if (lastMessage.tool_calls[0].name === "Response") {
    return "end";
  }
  // Otherwise we continue
  return "action";
};

// Define the function that calls the model
const callModel = async (state: { messages: BaseMessage[] }) => {
  const { messages } = state;
  const response = await boundModel.invoke(messages);
  // We return an object, because this will get added to the existing list
  return { messages: [response] };
};


## Define the graph


In [12]:
import { END, StateGraph } from "@langchain/langgraph";

// Define a new graph
const workflow = new StateGraph({
  channels: graphState,
});

// Define the two nodes we will cycle between
workflow.addNode("agent", callModel);
workflow.addNode("action", toolNode);

// Set the entrypoint as `agent`
// This means that this node is the first one called
workflow.setEntryPoint("agent");

// We now add a conditional edge
workflow.addConditionalEdges(
  // First, we define the start node. We use `agent`.
  // This means these are the edges taken after the `agent` node is called.
  "agent",
  // Next, we pass in the function that will determine which node is called next.
  route,
  {
    action: "action",
    end: END,
  },
);

// We now add a normal edge from `tools` to `agent`.
// This means that after `tools` is called, `agent` node is called next.
workflow.addEdge("action", "agent");

// Finally, we compile it!
// This compiles it into a LangChain Runnable,
// meaning you can use it as you would any other runnable
const app = workflow.compile();

StateGraph {
  nodes: {
    agent: RunnableLambda {
      lc_serializable: false,
      lc_kwargs: { func: [AsyncFunction: callModel] },
      lc_runnable: true,
      name: undefined,
      lc_namespace: [ "langchain_core", "runnables" ],
      func: [AsyncFunction: callModel]
    },
    action: ToolNode {
      lc_serializable: false,
      lc_kwargs: {},
      lc_runnable: true,
      name: "tools",
      lc_namespace: [ "langgraph" ],
      func: [Function: func],
      tags: undefined,
      config: { tags: [] },
      trace: true,
      recurse: true,
      tools: [
        DynamicStructuredTool {
          lc_serializable: false,
          lc_kwargs: [Object],
          lc_runnable: true,
          name: "search",
          verbose: false,
          callbacks: undefined,
          tags: [],
          metadata: {},
          returnDirect: false,
          description: "Call to surf the web.",
          func: [AsyncFunction: func],
          schema: [ZodObject]
        }
      ]
 

## Use it!

We can now use it! This now exposes the
[same interface](https://v02.api.js.langchain.com/classes/langchain_core_runnables.Runnable.html) as all
other LangChain runnables.

In [14]:
import { AIMessage, BaseMessage, HumanMessage } from "@langchain/core/messages";

const prettyPrint = (message: BaseMessage) => {
  let txt = `[${message._getType()}]: ${message.content}`;
  if (
    (message._getType() === "ai" &&
      (message as AIMessage)?.tool_calls?.length) ||
    0 > 0
  ) {
    const tool_calls = (message as AIMessage)?.tool_calls
      ?.map((tc) => `- ${tc.name}(${JSON.stringify(tc.args)})`)
      .join("\n");
    txt += ` \nTools: \n${tool_calls}`;
  }
  console.log(txt);
};

const inputs = {
  messages: [new HumanMessage("what is the weather in sf")],
};

for await (const output of await app.stream(inputs, { streamMode: "values" })) {
  const { messages } = output;
  prettyPrint(messages[messages.length - 1]);
  console.log("\n---\n");
}

[human]: what is the weather in sf

---

[ai]: [object Object] 
Tools: 
- search({"query":"weather in sf"})

---

[tool]: The answer to your question lies within.

---

[ai]: [object Object],[object Object] 
Tools: 
- Response({"temperature":65,"other_notes":"Sunny with a high of 70°F and a low of 55°F. Light winds around 10 mph."})

---

